In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import csv
from PIL import Image
import os
import librosa
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import cv2

from keras import layers
from keras.layers import Input, Add, Dense, Activation, Dropout, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import RMSprop,Adam
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
with open('../input/cassava-leaf-disease-classification/train.csv') as f:
    reader = csv.reader(f)
    next(reader, None)
    train_data = list(reader)

In [ ]:
train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
print((train_df.label.unique()))

In [ ]:
img = Image.open('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
img = np.array(img)
plt.imshow(img)
img.shape

In [ ]:
x_train = []
y_train = []
n = 0
dir = '../input/cassava-leaf-disease-classification/train_images/'
for data in train_data:
    #extracting labels and adding it to y_train
    label = int(data[1])
    label_array = np.zeros(5)
    label_array[label] = 1
    y_train.append(label_array)
    
    #creating array of images
    img_name = data[0]
    path = dir + str(img_name)
    img = cv2.imread(path)
    img = resize(img, (64, 64, 3))
    x_train.append(img)
    n+=1
    if (n%100 == 0):
        print('image processed ' + str(n) + ' of ' + str(len(train_data)))

In [ ]:
x_train = np.array(x_train)
print(x_train.shape)

In [ ]:
y_train = np.array(y_train)
y_train.shape

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32,kernel_size =(3,3), padding = 'Same', activation = 'relu', input_shape = (64,64,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(256, (4,4), padding='Same', activation='relu'))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train = x_train / 255

In [ ]:
model.fit(x_train, y_train, epochs = 50, batch_size = 128)

# **Predictions**

In [ ]:
test_dir = '../input/cassava-leaf-disease-classification/test_images/'
preds = [('image_id', 'label')]
for img in os.listdir(test_dir):
    path = test_dir + img
    image = cv2.imread(path)
    image = resize(image, (64,64,3))
    image = image.reshape(1, 64, 64, 3)
    y = model.predict(image)
    y = np.argmax(y)
    tup = (img, y)
    preds.append(tup)

In [ ]:
csvfile = open('submission.csv','w',newline='')
obj = csv.writer(csvfile)
for pred in preds:
    obj.writerow(pred)
csvfile.close()

In [ ]:
output1 = pd.read_csv('/kaggle/working/submission.csv')
output1